In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from trajectories.fitting import *
from trajectories.filtering import *
from visualization.graph import visualize_trajectory_graph
from visualization.video import *
from utils.video_utils import frame_generator

%load_ext autoreload
%autoreload 2

In [ ]:
# sources = ['60864', '60924']
sources = sorted(os.listdir('../volleyball_videos_small/'))

candidates = []
for source in sources:
    candidates.append(pd.read_csv(f'../volleyball_ball_annotation_small/{source}.txt', sep=' ', names=['x', 'y']).values)
candidates = np.concatenate(candidates)[:, np.newaxis, [1,0]] # concatenate sources and invert x and y
# candidates = np.concatenate([candidates, -np.ones(candidates.shape)], axis=1) # debug: add non detections
# candidates = np.concatenate([candidates, np.random.randint(0, 1280, (len(candidates), 50, 2))], axis=1) # debug: add non detections
candidates.shape

candidates = np.random.permutation(candidates.transpose(1,0,2)).transpose(1,0,2)

In [ ]:
import time

t0 = time.time()
fitting_info = fit_trajectories(candidates, seed_radius=60, d_threshold=20)
trajectory_graph = build_trajectory_graph(fitting_info)
shortest_paths = find_shortest_paths(trajectory_graph)
path_mapping = build_path_mapping(fitting_info, shortest_paths)
t1 = time.time()

t = t1-t0
print(t)
print(1/t)

In [ ]:
visualize_trajectory_graph(trajectory_graph, whole_graph=False)
plt.show()

In [ ]:
# sources_paths = f'../volleyball_videos/0/{sources}/'
# output_name = f'{sources}.mp4'
sources_paths = [f'../volleyball_videos_small/{src}/' for src in sources]
output_name = 'out.mp4'

create_trajectory_video(candidates, sources_paths, output_name, fitting_info, path_mapping, fps=30)

# Triggers

In [ ]:
seed_frames = [d['k_seed'] for d in fitting_info['trajectories']]

starting_frame = fitting_info['trajectories'][0]['k_seed']

velocities = []
v = np.array([0,0])
for frame_index in seed_frames:
    pm = path_mapping[frame_index]
    if pm is None:
        velocities.append(v)
        continue
    trajectory_info = fitting_info['trajectories'][pm]

    t = frame_index - trajectory_info['k_min']

    v0 = trajectory_info['v']
    a = trajectory_info['a']

    v = v0 + a*t
    velocities.append(v)

velocities = np.vstack(velocities)
accelerations = np.vstack([np.array([[0,0]]), velocities[1:]-velocities[:-1]])

In [ ]:
acceleration_angles = np.arctan2(accelerations[:,1], accelerations[:,0])/np.pi*180
acceleration_norms = np.linalg.norm(accelerations, axis=1)
norms_ratio = acceleration_norms[1:]/np.where(acceleration_norms[:-1]==0, 1, acceleration_norms[:-1])
norms_ratio = np.concatenate([[0], norms_ratio])

ag = 70
print(np.where(norms_ratio>5)[0])
print(np.where(np.abs(acceleration_angles)>ag)[0])

triggers = np.where((norms_ratio>5) & (np.abs(acceleration_angles)>ag))[0]
print(triggers)

In [ ]:
sources_paths = [f'../volleyball_videos_small/{src}/' for src in sources]
output_name = 'out_trig2.mp4'

create_trajectory_video(candidates, sources_paths, output_name, fitting_info, path_mapping, fps=30, trigger_frames=triggers)

In [ ]:
sources_paths = [f'../volleyball_videos_small/{src}/' for src in sources]
output_name = 'out_trig_clean2.mp4'

create_trajectory_video(candidates, sources_paths, output_name, fitting_info, path_mapping,
                        fps=30,
                        trigger_frames=triggers,
                        display=None,
                        num_prev=0,
                        num_next=0)

In [ ]:
sources_paths = 'raw_video.mp4'
output_name = 'out_complete3.mp4'

create_trajectory_video(candidates, sources_paths, output_name, fitting_info, path_mapping,
                        fps=30,
                        trigger_frames=triggers,
                        display=None,
                        num_prev=0,
                        num_next=0)

# Write images to video file

In [ ]:
from utils.video_utils import frame_generator
import cv2

out = cv2.VideoWriter(filename='poses.mp4',
                        fourcc=cv2.VideoWriter_fourcc(*'XVID'),
                        fps=30,
                        frameSize=(1280, 720))

for frame in frame_generator('poses'):
    out.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

out.release()

In [ ]:
if not os.path.exists('poses'):
    os.makedirs('poses')
for i, (frame, frame_pose) in enumerate(zip(frame_generator('raw_video.mp4'), frame_generator('raw_video_pose.mp4'))):
    pose = cv2.subtract(frame_pose, frame)
    plt.imsave(f'poses/{str(i).zfill(6)}.png', pose)